# Testing the deductron layer

This notebook implements one of the examples from the Keras documentation for sequence-to-sequence learning using RNNs. The task is to solve addition problems, formulated as strings. For example, the network should be able to take the string `341+78 ` and return the sum `419`.

This is accomplished using an encoding layer and a decoding layer. In the documentation example, both the encoder and the decoder are LSTMs. In this notebook, we generate some training data and train networks using both LSTMs and deductron layers.

The code for this example is taken from https://keras.io/examples/nlp/addition_rnn/ and lightly modified to fit our use case.

In [1]:
import keras
from keras import layers
import numpy as np
import deductron

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True        # Reverse the order of the input strings -- this seems to improve learning

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# Although a sigmoid is a natural choice for an activation with values in (0, 1), a ReLU
# constrained to produce output in [0, 1] has much better performance.
clipped_relu = lambda x: keras.activations.relu(x, max_value=1)

Using TensorFlow backend.


In [2]:
# Generate addition problems for training

class CharacterTable:
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """

    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.
        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.
        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[x] for x in x)


# All the numbers, plus sign and space for padding.
chars = "0123456789+ "
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
while len(questions) < TRAINING_SIZE:
    f = lambda: int(
        "".join(
            np.random.choice(list("0123456789"))
            for i in range(np.random.randint(1, DIGITS + 1))
        )
    )
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = "{}+{}".format(a, b)
    query = q + " " * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += " " * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)

In [3]:
# Vectorize the training and validation data

x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

## Reference example: LSTM encoder/decoder

The purpose of the next few sections is to compare the LSTM solution with a pure deductron solution as well as mixed LSTM-deductron solutions.

The reference approach uses two layers, thinking of the first as an encoder and the second as a decoder. The first layer extracts features from the input and produces an internal representation of the problem. The final output of this "encoder" layer is then repeated to form a constant sequence and passed to the "decoder" layer as its input.

Following the existing example, we begin by building a model in which each of the encoder and decoder is a single LSTM layer with 128 hidden units. Further examples will replace each of these layers in turn with a deductron layer (also with 128 units).

In [4]:
model = keras.Sequential()
# "Encode" the input sequence using a LSTM, producing an output of size 128.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
# Replacing the encoder with a deductron layer seems to hurt accuracy.
model.add(layers.LSTM(128, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# We'll use a single layer for the decoder -- adding more layers doesn't seem to improve results
model.add(layers.LSTM(128, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.Dense(len(chars), activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
dense_1 (Dense)              (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


Now it is time to train the model. We will train all of the models for 12 epochs (more would be better, but the training is a little time consuming). After each epoch we visualize 10 samples from the validation set so that we can see the sort of output we are getting.

In [5]:
epochs = 12
batch_size = 32

# Train the model each generation and show predictions against the validation
# dataset.
for epoch in range(0, epochs):
    print()
    print("Iteration", epoch)
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=1,
        validation_data=(x_val, y_val),
    )
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print("Q", q[::-1] if REVERSE else q, end=" ")
        print("T", correct, end=" ")
        if correct == guess:
            print("☑ " + guess)
        else:
            print("☒ " + guess)


Iteration 0
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 27s 608us/step - loss: 1.7634 - accuracy: 0.3535 - val_loss: 1.6066 - val_accuracy: 0.3937
Q 422+25  T 447  ☒ 358 
Q 285+47  T 332  ☒ 587 
Q 69+535  T 604  ☒ 622 
Q 17+475  T 492  ☒ 587 
Q 198+98  T 296  ☒ 992 
Q 185+992 T 1177 ☒ 1247
Q 360+83  T 443  ☒ 732 
Q 37+308  T 345  ☒ 387 
Q 26+472  T 498  ☒ 387 
Q 22+196  T 218  ☒ 227 

Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 23s 511us/step - loss: 1.3473 - accuracy: 0.4950 - val_loss: 1.1510 - val_accuracy: 0.5706
Q 88+936  T 1024 ☒ 1010
Q 274+67  T 341  ☒ 346 
Q 613+16  T 629  ☒ 634 
Q 871+2   T 873  ☒ 881 
Q 243+472 T 715  ☒ 740 
Q 508+6   T 514  ☒ 510 
Q 616+5   T 621  ☒ 614 
Q 583+94  T 677  ☒ 661 
Q 58+323  T 381  ☒ 396 
Q 586+14  T 600  ☒ 510 

Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===============

Typically, after 12 epochs, the above network should reach about 98-99% accuracy, with most visualized problems solved correctly.

## A pure deductron implementation

Our first change is to replace both the encoder and decoder LSTM layers with deductron layers.

In [6]:
model = keras.Sequential()
model.add(deductron.Deductron(128, activation = clipped_relu, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
model.add(deductron.Deductron(128, activation = clipped_relu, return_sequences=True))

model.add(layers.Dense(len(chars), activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
deductron_1 (Deductron)      (None, 128)               19840     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
deductron_2 (Deductron)      (None, 4, 128)            49536     
_________________________________________________________________
dense_2 (Dense)              (None, 4, 12)             1548      
Total params: 70,924
Trainable params: 70,924
Non-trainable params: 0
_________________________________________________________________


In [7]:
for epoch in range(epochs):
    print()
    print("Iteration", epoch)
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=1,
        validation_data=(x_val, y_val),
    )
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print("Q", q[::-1] if REVERSE else q, end=" ")
        print("T", correct, end=" ")
        if correct == guess:
            print("☑ " + guess)
        else:
            print("☒ " + guess)


Iteration 0
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 38s 846us/step - loss: 1.8258 - accuracy: 0.3457 - val_loss: 1.7251 - val_accuracy: 0.3729
Q 875+299 T 1174 ☒ 1466
Q 870+71  T 941  ☒ 189 
Q 984+32  T 1016 ☒ 122 
Q 85+886  T 971  ☒ 144 
Q 868+53  T 921  ☒ 133 
Q 11+56   T 67   ☒ 16  
Q 70+887  T 957  ☒ 164 
Q 609+771 T 1380 ☒ 1144
Q 900+77  T 977  ☒ 176 
Q 8+638   T 646  ☒ 13  

Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 37s 817us/step - loss: 1.6641 - accuracy: 0.3972 - val_loss: 1.6397 - val_accuracy: 0.4004
Q 14+48   T 62   ☒ 428 
Q 0+525   T 525  ☒ 556 
Q 81+29   T 110  ☒ 120 
Q 748+943 T 1691 ☒ 1222
Q 332+829 T 1161 ☒ 1266
Q 39+61   T 100  ☒ 120 
Q 781+995 T 1776 ☒ 1666
Q 44+3    T 47   ☑ 47  
Q 926+9   T 935  ☒ 900 
Q 637+56  T 693  ☒ 722 

Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===============

The pure deductron network does improve accuracy with training, but learns much more slowly, reaching only around 58-60% accuracy after 12 epohcs. Visual inspection of the output shows many repeated digits and similar apparent patterns, possibly pointing to insufficient feature extraction.

## Mixed solutions

In [8]:
# LSTM encoder, deductron decoder
model = keras.Sequential()
model.add(layers.LSTM(128, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))

model.add(deductron.Deductron(128, activation = clipped_relu, return_sequences=True))

model.add(layers.Dense(len(chars), activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
deductron_3 (Deductron)      (None, 4, 128)            49536     
_________________________________________________________________
dense_3 (Dense)              (None, 4, 12)             1548      
Total params: 123,276
Trainable params: 123,276
Non-trainable params: 0
_________________________________________________________________


In [9]:
for epoch in range(epochs):
    print()
    print("Iteration", epoch)
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=1,
        validation_data=(x_val, y_val),
    )
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print("Q", q[::-1] if REVERSE else q, end=" ")
        print("T", correct, end=" ")
        if correct == guess:
            print("☑ " + guess)
        else:
            print("☒ " + guess)


Iteration 0
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 33s 733us/step - loss: 1.8104 - accuracy: 0.3410 - val_loss: 1.6040 - val_accuracy: 0.4069
Q 3+874   T 877  ☒ 998 
Q 535+674 T 1209 ☒ 1064
Q 4+204   T 208  ☒ 444 
Q 85+692  T 777  ☒ 905 
Q 45+25   T 70   ☒ 14  
Q 857+30  T 887  ☒ 995 
Q 85+996  T 1081 ☒ 1005
Q 674+69  T 743  ☒ 765 
Q 95+72   T 167  ☒ 100 
Q 23+54   T 77   ☒ 12  

Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 28s 619us/step - loss: 1.4378 - accuracy: 0.4794 - val_loss: 1.3240 - val_accuracy: 0.5109
Q 21+275  T 296  ☒ 255 
Q 280+802 T 1082 ☒ 100 
Q 663+840 T 1503 ☒ 1555
Q 360+39  T 399  ☒ 395 
Q 846+14  T 860  ☒ 855 
Q 366+903 T 1269 ☒ 1225
Q 0+170   T 170  ☒ 110 
Q 666+348 T 1014 ☒ 900 
Q 45+36   T 81   ☒ 75  
Q 412+722 T 1134 ☒ 1155

Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===============

Accuracy with the LSTM encoder and deductron decoder is comparable to using LSTM in both roles. This adds credibility to the idea that the problem in the previous test was insufficient feature extraction by the deductron layer.

Reversing the roles of the LSTM and deductron adds more weight to this idea.

In [10]:
# Deductron encoder, LSTM decoder
num_layers = 1  # Number of layers in the decoder.
model = keras.Sequential()
model.add(deductron.Deductron(128, activation = clipped_relu, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))

model.add(layers.LSTM(128, return_sequences=True))

model.add(layers.Dense(len(chars), activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
deductron_4 (Deductron)      (None, 128)               19840     
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
dense_4 (Dense)              (None, 4, 12)             1548      
Total params: 152,972
Trainable params: 152,972
Non-trainable params: 0
_________________________________________________________________


In [11]:
for epoch in range(epochs):
    print()
    print("Iteration", epoch)
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=1,
        validation_data=(x_val, y_val),
    )
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print("Q", q[::-1] if REVERSE else q, end=" ")
        print("T", correct, end=" ")
        if correct == guess:
            print("☑ " + guess)
        else:
            print("☒ " + guess)


Iteration 0
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 38s 845us/step - loss: 1.7698 - accuracy: 0.3565 - val_loss: 1.6711 - val_accuracy: 0.3809
Q 674+9   T 683  ☒ 161 
Q 842+361 T 1203 ☒ 101 
Q 3+665   T 668  ☒ 611 
Q 461+7   T 468  ☒ 111 
Q 756+80  T 836  ☒ 655 
Q 28+219  T 247  ☒ 211 
Q 657+47  T 704  ☒ 601 
Q 42+73   T 115  ☒ 446 
Q 88+114  T 202  ☒ 906 
Q 704+521 T 1225 ☒ 101 

Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 33s 738us/step - loss: 1.6271 - accuracy: 0.4014 - val_loss: 1.5986 - val_accuracy: 0.4085
Q 932+18  T 950  ☒ 301 
Q 61+190  T 251  ☒ 101 
Q 175+8   T 183  ☒ 12  
Q 87+669  T 756  ☒ 855 
Q 762+23  T 785  ☒ 394 
Q 499+161 T 660  ☒ 104 
Q 646+8   T 654  ☒ 141 
Q 435+73  T 508  ☒ 411 
Q 3+273   T 276  ☒ 340 
Q 965+85  T 1050 ☒ 104 

Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===============

This network has only marginally better performance than the pure deductron, suggesting that the low performance was tied specifically to the deductron in the encoder role, not the absence of an LSTM layer in the pure deductron network.

## Discussion

The above tests show that the deductron architecture is capable of learning the sequence-to-sequence addition task. However, in the encoder-decoder network structure, the deductron does not perform as well as LSTM in the "encoder" role. Using LSTM for the encoder and deductron for the decoder reaches approximate parity with the LSTM-LSTM network; reversing the roles leads to slower learning. This suggests that the deductron may not have enough complexity to extract the necessary features for this task, but is effective at using these features once learned by the LSTM layer.